## **Applio NoUI**
A simple, high-quality voice conversion tool focused on ease of use and performance.

[Support](https://discord.gg/urxFjYmYYh) — [GitHub](https://github.com/IAHispano/Applio) — [Terms of Use](https://github.com/IAHispano/Applio/blob/main/TERMS_OF_USE.md)

<br>

---

<br>

#### **Acknowledgments**

To all external collaborators for their special help in the following areas: Hina (Encryption method), Poopmaster (Extra section), Shirou (UV installer),  Kit Lemonfoot (NoUI inspiration)

#### **Disclaimer**
By using Applio, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Applio disclaims liability and reserves the right to amend these terms.

# <font class="markdown-google-sans">**Install Applio**</font>
> If the runtime restarts, re-run the installation steps.

In [ ]:
#@title ## <font class="markdown-google-sans">Mount Google Drive</font>
from google.colab import drive

drive.mount("/content/drive", force_remount = True)

In [ ]:
#@title ## <font class="markdown-google-sans">Setup runtime environment</font> `(01m 52s)`
import os
from multiprocessing import cpu_count

CPU_Cores = cpu_count()
post_process = False
hop_length = 128 # Common default value for hop_length, added for consistency across cells

!git config --global advice.detachedHead false
!git clone https://github.com/kubinka0505/Applio
os.chdir("/content/Applio")

print()

!sudo update-alternatives --set python3 /usr/bin/python3.10
!curl -LsSf https://astral.sh/uv/install.sh | sh

print()

!sudo apt-get install aria2

print("Installing requirements...")
!uv pip install -q -r requirements.txt

print()

print("Finished installing requirements!")
!python core.py "prerequisites" --models "True" --pretraineds_hifigan "True"

### **Infer**


In [ ]:
# @title Download model
# @markdown Hugging Face or Google Drive
model_link = "https://huggingface.co/Darwin/Darwin/resolve/main/Darwin.zip"  # @param {type:"string"}

%cd /content/Applio
!python core.py download --model_link "{model_link}"

In [ ]:
# @title Run Inference
# @markdown Please upload the audio file to your Google Drive path `/content/drive/MyDrive` and specify its name here. For the model name, use the zip file name without the extension. Alternatively, you can check the path `/content/Applio/logs` for the model name (name of the folder).
%cd /content/Applio
from pathlib import Path

model_name = "Darwin"  # @param {type:"string"}
model_path = Path(f"/content/Applio/logs/{model_name}")
if not (model_path.exists() and model_path.is_dir()):
    raise FileNotFoundError(f"Model directory not found: {model_path.resolve()}")

# Select either the last checkpoint or the final weight
!ls -t "{model_path}"/"{model_name}"_*e_*s.pth "{model_path}"/"{model_name}".pth 2> /dev/null | head -n 1 > /tmp/pth.txt
pth_file = open("/tmp/pth.txt", "r").read().strip()
if pth_file == "":
    raise FileNotFoundError(f"No model weight found in directory: {model_path.resolve()}
Make sure that the file is properly named (e.g. "{model_name}.pth)"")

!ls -t "{model_path}"/*.index | head -n 1 > /tmp/index.txt
index_file = open("/tmp/index.txt", "r").read().strip()

input_path = "/content/example.wav"  # @param {type:"string"}
output_path = "/content/output.wav"
export_format = "WAV"  # @param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A'] {allow-input: false}
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
f0_up_key = 0  # @param {type:"slider", min:-24, max:24, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
f0_autotune = False  # @param{type:"boolean"}
formant_shift = False # @param{type:"boolean"}
formant_qfrency = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
formant_timbre = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
embedder_model = "contentvec" # @param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" # @param {type:"string"}

!rm -f "{output_path}"
if post_process:
  !python core.py infer --pitch "{f0_up_key}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --hop_length "{hop_length}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}" --post_process "{post_process}" --reverb "{reverb}" --pitch_shift "{pitch_shift}" --limiter "{limiter}" --gain "{gain}" --distortion "{distortion}" --chorus "{chorus}" --bitcrush "{bitcrush}" --clipping "{clipping}" --compressor "{compressor}" --delay "{delay}" --reverb_room_size "{reverb_room_size}" --reverb_damping "{reverb_damping}" --reverb_wet_gain "{reverb_wet_gain}" --reverb_dry_gain "{reverb_dry_gain}" --reverb_width "{reverb_width}" --reverb_freeze_mode "{reverb_freeze_mode}" --pitch_shift_semitones "{pitch_shift_semitones}" --limiter_threshold "{limiter_threshold}" --limiter_release_time "{limiter_release_time}" --gain_db "{gain_db}" --distortion_gain "{distortion_gain}" --chorus_rate "{chorus_rate}" --chorus_depth "{chorus_depth}" --chorus_center_delay "{chorus_center_delay}" --chorus_feedback "{chorus_feedback}" --chorus_mix "{chorus_mix}" --bitcrush_bit_depth "{bitcrush_bit_depth}" --clipping_threshold "{clipping_threshold}" --compressor_threshold "{compressor_threshold}" --compressor_ratio "{compressor_ratio}" --compressor_attack "{compressor_attack}" --compressor_release "{compressor_release}" --delay_seconds "{delay_seconds}" --delay_feedback "{delay_feedback}" --delay_mix "{delay_mix}"
else:
  !python core.py infer --pitch "{f0_up_key}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}" --post_process "{post_process}"

if Path(output_path).exists():
  from IPython.display import Audio, display
  output_path = output_path.replace(".wav", f".{export_format.lower()}")
  display(Audio(output_path, autoplay=True))

In [ ]:
# @title Enable post-processing effects for inference
post_process = False # @param{type:"boolean"}
reverb = False # @param{type:"boolean"}
pitch_shift = False # @param{type:"boolean"}
limiter = False # @param{type:"boolean"}
gain = False # @param{type:"boolean"}
distortion = False # @param{type:"boolean"}
chorus = False # @param{type:"boolean"}
bitcrush = False # @param{type:"boolean"}
clipping = False # @param{type:"boolean"}
compressor = False # @param{type:"boolean"}
delay = False # @param{type:"boolean"}

reverb_room_size = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_damping = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_wet_gain = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}
reverb_dry_gain = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}
reverb_width = 1.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_freeze_mode = 0.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

pitch_shift_semitones = 0.0 # @param {type:"slider", min:-12.0, max:12.0, step:0.1}

limiter_threshold = -1.0 # @param {type:"slider", min:-20.0, max:0.0, step:0.1}
limiter_release_time = 0.05 # @param {type:"slider", min:0.0, max:1.0, step:0.01}

gain_db = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}

distortion_gain = 0.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

chorus_rate = 1.5 # @param {type:"slider", min:0.1, max:10.0, step:0.1}
chorus_depth = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
chorus_center_delay = 15.0 # @param {type:"slider", min:0.0, max:50.0, step:0.1}
chorus_feedback = 0.25 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
chorus_mix = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

bitcrush_bit_depth = 4 # @param {type:"slider", min:1, max:16, step:1}

clipping_threshold = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

compressor_threshold = -20.0 # @param {type:"slider", min:-60.0, max:0.0, step:0.1}
compressor_ratio = 4.0 # @param {type:"slider", min:1.0, max:20.0, step:0.1}
compressor_attack = 0.001 # @param {type:"slider", min:0.0, max:0.1, step:0.001}
compressor_release = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.01}

delay_seconds = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
delay_feedback = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
delay_mix = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}


# <font class="markdown-google-sans">**Train**</font>

In [ ]:
#@title ## <font class="markdown-google-sans">Setup model variables</font>
model_name = "kubinka0505" #@param {type: "string"}
sample_rate = "48k" #@param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip("k")) * 1000

In [ ]:
#@title ## <font class="markdown-google-sans">Preprocess Dataset</font>
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
dataset_path = "/content/drive/MyDrive/datasets/kubinka0505_Normal" #@param {type:"string"}

cut_preprocess = "Automatic" #@param ["Skip", "Simple", "Automatic"]
chunk_len = 3 #@param {type: "slider", min: 0.5, max: 5.0, step: 0.5}
overlap_len = 0.3 #@param {type: "slider", min: 0, max: 0.5, step: 0.1}
process_effects = False #@param{type: "boolean"}
noise_reduction = False #@param{type: "boolean"}
noise_reduction_strength = 0.7 #@param {type: "slider", min: 0, max:1, step: 0.1}

os.chdir("/content/Applio")

!python core.py preprocess \
    --model_name "{model_name}" \
    --dataset_path "{dataset_path}" \
    --sample_rate "{sr}" \
    --cpu_cores "{CPU_Cores}" \
    --cut_preprocess "{cut_preprocess}" \
    --process_effects "{process_effects}" \
    --noise_reduction "{noise_reduction}" \
    --noise_reduction_strength "{noise_reduction_strength}" \
    --chunk_len "{chunk_len}" \
    --overlap_len "{overlap_len}"

In [ ]:
#@title ## <font class="markdown-google-sans">Extract Features</font> `(00m 30s)`
import os

f0_method = "rmvpe" #@param ["crepe", "crepe-tiny", "rmvpe"] {allow-input: false}

sr = int(sample_rate.rstrip("k")) * 1000
include_mutes = 2 #@param {type: "slider", min: 0, max: 10, step: 1}
embedder_model = "contentvec" #@param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" #@param {type: "string"}

os.chdir("/content/Applio")

!python core.py extract \
    --model_name "{model_name}" \
    --f0_method "{f0_method}" \
    --hop_length "{hop_length}" \
    --sample_rate "{sr}" \
    --cpu_cores "{CPU_Cores}" \
    --gpu "0" \
    --embedder_model "{embedder_model}" \
    --embedder_model_custom "{embedder_model_custom}" \
    --include_mutes "{include_mutes}"

In [ ]:
#@title ## <font class="markdown-google-sans">Generate index file</font> `(00m 16s)`
import os

index_algorithm = "Auto" #@param ["Auto", "Faiss", "KMeans"] {allow-input: false}

os.chdir("/content/Applio")

!python core.py index \
    --model_name "{model_name}" \
    --index_algorithm "{index_algorithm}"

In [ ]:
#@title ## <font class="markdown-google-sans">Train</font>
import os
from pathlib import Path

# ========================
# 🔧 Configuration Section
# ========================

# AutoBackup Parameters
cooldown = 1 #@param {type: "slider", min: 0, max: 100, step: 0}
auto_backups = False #@param {type: "boolean"}

#@markdown <br>

#@markdown Training Settings
total_epoch = 20 #@param {type: "integer"}
batch_size = 8 #@param {type: "slider", min: 1, max: 25, step: 0}
gpu = 0
sr = int(sample_rate.rstrip("k")) * 1000
pretrained = True #@param {type: "boolean"}
cleanup = False #@param {type: "boolean"}
cache_data_in_gpu = False #@param {type: "boolean"}
vocoder = "HiFi-GAN" #@param ["HiFi-GAN"]
checkpointing = False
tensorboard = True #@param {type: "boolean"}

#@markdown <br>

#@markdown Save Behavior
save_every_epoch = 5 #@param {type: "slider", min: 1, max: 100, step: 0}
save_only_latest = False #@param {type: "boolean"}
save_every_weights = True #@param {type: "boolean"}

#@markdown <br>

#@markdown Overtraining Detector
overtraining_detector = False #@param {type:"boolean"}
overtraining_threshold = 50 #@param {type: "slider", min: 1, max: 100, step: 0}

#@markdown <br>

#@markdown Pretrained Options
custom_pretrained = True #@param{type:"boolean"}
g_pretrained_path = "http://hf.co/blaise-tk/TITAN/resolve/main/models/medium/48k/pretrained/G-f048k-TITAN-Medium.pth" #@param {type:"string"}
d_pretrained_path = "http://hf.co/blaise-tk/TITAN/resolve/main/models/medium/48k/pretrained/D-f048k-TITAN-Medium.pth" #@param {type:"string"}

# ======================
# 📁 Path Configuration
# ======================

LOGS_FOLDER = "/content/Applio/logs/"
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/RVC_Backup"

# ========================
# 🧠 Global State Defaults
# ========================

if "autobackups" not in globals():
    autobackups = False
if "pretrained" not in globals():
    pretrained = True
if "custom_pretrained" not in globals():
    custom_pretrained = False
if "g_pretrained_path" not in globals():
    g_pretrained_path = "Custom Path"
if "d_pretrained_path" not in globals():
    d_pretrained_path = "Custom Path"

# ====================
# 📦 Import Functions
# ====================

def import_google_drive_backup():
    print("Importing Google Drive backup...")
    google_drive_path = Path(GOOGLE_DRIVE_PATH)
    logs_folder = Path(LOGS_FOLDER)

    for root, dirs, files in os.walk(google_drive_path):
        root_path = Path(root)

        for filename in files:
            filepath = root_path / filename

            if filepath.is_file():
                rel_path = filepath.relative_to(google_drive_path)
                backup_filepath = logs_folder / rel_path
                backup_folderpath = backup_filepath.parent

                if not backup_folderpath.exists():
                    backup_folderpath.mkdir(parents=True)
                    print(f"Created backup folder: {backup_folderpath}", flush=True)

                shutil.copy2(filepath, backup_filepath)
                print(f"Imported file from Google Drive backup: {filename}")

    print("Google Drive backup import completed.")

def backup_files(logs_dir: str, drive_pth: str):
    print("\nStarting backup loop...")

    logs_folder = Path(logs_folder)
    drive_path = Path(drive_path)

    last_backup_timestamps_path = logs_folder / "last_backup_timestamps.txt"
    fully_updated = False

    while True:
        try:
            updated_files = 0
            deleted_files = 0
            new_files = 0
            last_backup_timestamps = {}

            try:
                with last_backup_timestamps_path.open("r") as f:
                    last_backup_timestamps = dict(line.strip().split(":", 1) for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(logs_folder):
                # Convert to Path objects
                root_path = Path(root)

                # Remove unwanted subdirs from walk
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")

                for filename in files:
                    if filename == "last_backup_timestamps.txt":
                        continue

                    filepath = root_path / filename
                    if not filepath.is_file():
                        continue

                    rel_path = filepath.relative_to(logs_folder)
                    backup_filepath = drive_path / rel_path
                    backup_folderpath = backup_filepath.parent

                    if not backup_folderpath.exists():
                        backup_folderpath.mkdir(parents=True)

                    last_backup_timestamp = last_backup_timestamps.get(str(filepath))
                    current_timestamp = filepath.stat().st_mtime

                    if last_backup_timestamp is None or float(last_backup_timestamp) < current_timestamp:
                        shutil.copy2(filepath, backup_filepath)
                        last_backup_timestamps[str(filepath)] = str(current_timestamp)

                        if last_backup_timestamp is None:
                            new_files += 1
                        else:
                            updated_files += 1

            # Handle deletions
            for filepath_str in list(last_backup_timestamps.keys()):
                filepath = Path(filepath_str)

                if not filepath.exists():
                    rel_path = filepath.relative_to(logs_folder)
                    backup_filepath = drive_path / rel_path

                    if backup_filepath.exists():
                        backup_filepath.unlink()
                        deleted_files += 1

                    del last_backup_timestamps[filepath_str]

            if updated_files > 0 or deleted_files > 0 or new_files > 0:
                print(f"Backup Complete: {new_files} new, {updated_files} updated, {deleted_files} deleted.")
                fully_updated = False
            elif not fully_updated:
                print("Files are up to date.")
                fully_updated = True

            with last_backup_timestamps_path.open("w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(cooldown if fully_updated else 0.1)

        except Exception as error:
            print(f"An error occurred during backup: {error}")


def start_train():
    if tensorboard:
        %load_ext tensorboard
        %tensorboard --logdir /content/Applio/logs/

    os.chdir("/content/Applio")
    !python core.py train \
        --model-name "{model_name}" \
        --save-every-epoch "{save_every_epoch}" \
        --save-only-latest "{save_only_latest}" \
        --save-every-weights "{save_every_weights}" \
        --total-epoch "{total_epoch}" \
        --sample-rate "{sr}" \
        --batch-size "{batch_size}" \
        --gpu "{gpu}" \
        --pretrained "{pretrained}" \
        --custom-pretrained "{custom_pretrained}" \
        --g-pretrained-path "{g_pretrained_path}" \
        --d-pretrained-path "{d_pretrained_path}" \
        --overtraining-detector "{overtraining_detector}" \
        --overtraining-threshold "{overtraining_threshold}" \
        --cleanup "{cleanup}" \
        --cache-data-in-gpu "{cache_data_in_gpu}" \
        --vocoder "{vocoder}" \
        --checkpointing "{checkpointing}"

# ============================
# 🚀 Start Threads and Loops
# ============================

if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")

server_thread = threading.Thread(target=start_train)
server_thread.start()

if auto_backups:
    backup_files(LOGS_FOLDER, GOOGLE_DRIVE_PATH)

In [ ]:
#@title ## <font class="markdown-google-sans">Export model</font>
from pathlib import Path
import zipfile
import shutil

export_for = "Inference" #@param ["Training", "Inference"] {allow-input: false}

logs_folder = Path(f"/content/Applio/logs/{model_name}/")
if not (logs_folder.exists() and logs_folder.is_dir()):
    raise FileNotFoundError(f"{model_name} model folder not found")

export_zip_path = Path(f"/content/{model_name}.zip")

#-=-=-=-#

if export_for.lower().startswith("train"):
    # Zip the entire model folder
    with zipfile.ZipFile(export_zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
        for file in logs_folder.rglob("*"):
            if file.is_file():
                zipf.write(file, arcname = file.relative_to(logs_folder.parent))
else:
    # Inference mode: find latest weight file
    model_files = sorted(
        (logs_folder.glob(f"{model_name}_*e_*s.pth")),
        key = lambda f: f.stat().st_mtime,
        reverse = True
    )
    if not model_files:
        raise FileNotFoundError("Model has no weight file, please finish training first")

    weight_path = model_files[0]
    weight_name = weight_path.name
    index_file = logs_folder / f"{model_name}.index"

    with zipfile.ZipFile(export_zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(weight_path, arcname = f"{model_name}/{weight_name}")
        if index_file.exists():
            zipf.write(index_file, arcname = f"{model_name}/{index_file.name}")

#-=-=-=-#

# Move to Google Drive if mounted
backup_path = Path("/content/drive/MyDrive/RVC_Backup/")

if Path("/content/drive").is_mount():
    backup_path.mkdir(parents = True, exist_ok = True)
    shutil.move(str(export_zip_path), str(backup_path / export_zip_path.name))
    print(f"Exported model as {backup_path / export_zip_path.name}")
else:
    print(f"Drive not mounted, exporting model as {export_zip_path}")

### **Resume training**

In [ ]:
# @title Load a Backup
from google.colab import drive
import os
import shutil

# @markdown Put the exact name you put as your Model Name in Applio.
modelname = "My-Project"  # @param {type:"string"}
source_path = "/content/drive/MyDrive/RVC_Backup/" + modelname
destination_path = "/content/Applio/logs/" + modelname
backup_timestamps_file = "last_backup_timestamps.txt"
if not os.path.exists(source_path):
    print(
        "The model folder does not exist. Please verify the name is correct or check your Google Drive."
    )
else:
    time_ = os.path.join("/content/drive/MyDrive/RVC_Backup/", backup_timestamps_file)
    time__ = os.path.join("/content/Applio/logs/", backup_timestamps_file)
    if os.path.exists(time_):
        shutil.copy(time_, time__)
    shutil.copytree(source_path, destination_path)
    print("Model backup loaded successfully.")

In [ ]:
# @title Set training variables
# @markdown ### ➡️ Use the same as you did previously
model_name = "Darwin"  # @param {type:"string"}
sample_rate = "40k"  # @param ["32k", "40k", "48k"] {allow-input: false}
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe"] {allow-input: false}
sr = int(sample_rate.rstrip("k")) * 1000